In [10]:
import pytorch_lightning
from pytorchvideo.data import LabeledVideoDataset, UniformClipSampler, RandomClipSampler
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, Lambda, RandomCrop, RandomHorizontalFlip
from transforms import (
    ApplyTransformToKey,
    UniformTemporalSubsample,
    Normalize,
    RandomShortSideScale,
)

In [11]:
import csv 
def read_csv_to_list_of_tuples(filename: str):
    data = []

    with open(filename, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            # Assuming the first column contains file paths and the second column contains labels
            filepath = row[0]
            label = row[1]
            data.append((filepath, {"label": int(label)}))

    return data

In [12]:
class DataModule(pytorch_lightning.LightningDataModule):
    def __init__(self, clip_duration=60, batch_size=8, num_workers=8) -> None:
        super().__init__()
        self.CLIP_DURATION = clip_duration
        self.BATCH_SIZE = batch_size
        self.NUM_WORKERS = num_workers
        self.transform = Compose(
            [
                ApplyTransformToKey(
                    key="video",
                    transform=Compose(
                        [
                            UniformTemporalSubsample(8),
                            Lambda(lambda x: x / 255.0),
                            Normalize((0.45, 0.45, 0.45),
                                      (0.225, 0.225, 0.225)),
                            RandomShortSideScale(min_size=256, max_size=320),
                            RandomCrop(244),
                            RandomHorizontalFlip(p=0.5),
                        ]
                    ),
                ),
            ]
        )

    def train_dataloader(self):
        train_dataset = LabeledVideoDataset(
            labeled_video_paths=read_csv_to_list_of_tuples("train.csv"),
            clip_sampler=UniformClipSampler(self.CLIP_DURATION),
            decode_audio=False,
            transform=self.transform
        )

        return DataLoader(dataset=train_dataset, batch_size=self.BATCH_SIZE)

    def val_dataloader(self):
        val_dataset = LabeledVideoDataset(
            labeled_video_paths=read_csv_to_list_of_tuples("val.csv"),
            clip_sampler=UniformClipSampler(self.CLIP_DURATION),
            decode_audio=False,
            transform=self.transform
        )

        return DataLoader(dataset=val_dataset, batch_size=self.BATCH_SIZE)

In [13]:
import timm
model_name = 'inception_v4'
model = timm.create_model(model_name, pretrained=False)

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [15]:
import pytorchvideo.models.resnet

def make_kinetics_resnet():
  return pytorchvideo.models.resnet.create_resnet(
      input_channel=3,
      model_depth=50,
      model_num_class=400,
      norm=nn.BatchNorm3d,
      activation=nn.ReLU,
  )

In [16]:
class VideoClassificationLightningModule(pytorch_lightning.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = make_kinetics_resnet()

  def forward(self, x):
    return self.model(x)

  def training_step(self, batch, batch_idx):
    y_hat = self.model(batch["video"])
    loss = F.cross_entropy(y_hat, torch.Tensor(batch["label"]))
    self.log("train_loss", loss.item())

    return loss
  
  def validation_step(self, batch, batch_idx):
      y_hat = self.model(batch["video"])
      loss = F.cross_entropy(y_hat, torch.tensor(batch["label"]))
      self.log("val_loss", loss)
      return loss

  def configure_optimizers(self):
      return torch.optim.Adam(self.parameters(), lr=1e-1)

In [17]:
classification_module = VideoClassificationLightningModule()
data_module = DataModule()
trainer = pytorch_lightning.Trainer(max_epochs=10)
trainer.fit(classification_module, data_module)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type | Params
-------------------------------
0 | model | Net  | 32.5 M
-------------------------------
32.5 M    Trainable params
0         Non-trainable params
32.5 M    Total params
129.892   Total estimated model params size (MB)


Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:03<00:03,  0.28it/s]

C:\Users\Andreas\AppData\Local\Temp\ipykernel_49744\2736581558.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = F.cross_entropy(y_hat, torch.tensor(batch["label"]))
c:\Users\Andreas\anaconda3\envs\SSBD\lib\site-packages\pytorch_lightning\utilities\data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 8. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 15925248000 bytes.